## Since notebook involves training the BERT (to be specific - RoBERTa model) we can't run it again since the first run took 16h :)

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading the file to pandas
df = pd.read_csv('Dataset/Twitter/train.csv', encoding = "ISO-8859-1", header = None)[[0, 5]].rename(columns = {0: 'target', 5: 'text'})

# Mapping the target to 0 and 1
df['target'] = np.where(df['target']==4, 0, 1)
df.head()

,target,text
0,1,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,is upset that he can't update his Facebook by ...
2,1,@Kenichan I dived many times for the ball. Man...
3,1,my whole body feels itchy and like its on fire
4,1,"@nationwideclass no, it's not behaving at all...."


In [3]:
from sklearn.model_selection import train_test_split

# We fine tuned the model on small sample of the data and then trained the model on the entire dataset
# df1 = df[775000:-775000]
df1 = df
df1.head()

,target,text
0,1,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,is upset that he can't update his Facebook by ...
2,1,@Kenichan I dived many times for the ball. Man...
3,1,my whole body feels itchy and like its on fire
4,1,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Here, we can see that the dataset is perfectly distributed, i.e., no issues of imbalanced dataset|
df1['target'].value_counts()

1    800000
0    800000
Name: target, dtype: int64

In [5]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [6]:
def custom_standardization(input_data):
    import preprocessor as p
    processed_data = p.clean(input_data)
    lowercase_value = processed_data.lower()
    return lowercase_value

# Since we're using transfer learning, we have to use same format of data which was used for training
# Thus, with regards to pre-processing - we're using regex to pre-process and converting to lower case.
# Once the data has been processed we can pass it to model for training

In [7]:
df1["text"] = df1["text"].parallel_apply(custom_standardization)
df1.reset_index(drop = True, inplace = True)

In [8]:
max(df1['text'].str.split().apply(lambda x: len(x)))

64

# Train-Val Split

In [9]:
# Doing the train test split for creating a training and validation dataset
train_df, test_df = train_test_split(df1, test_size=0.2, random_state = 99)

# Setup for training

In [10]:
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel


# We're using a base model for re-training with the base model trained on depression dataset
tokenizer = AutoTokenizer.from_pretrained("paulagarciaserrano/roberta-depression-detection")

labels = {0: 0, 1: 1}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['target']]
        
        # Setting the parameters for the BERT tokenizer with max lenght = 64 and converting them to pytorch tensors
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 64, truncation=True,
                                return_tensors="pt") for text in tqdm(df['text'])]
        
    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [11]:
from torch import nn
from transformers import BertConfig, BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.1):
        
        super(BertClassifier, self).__init__()
        
        self.bert = AutoModel.from_pretrained("paulagarciaserrano/roberta-depression-detection")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 192)
        self.linear1 = nn.Linear(192, 64)
        self.linear2 = nn.Linear(64, 2)
        self.gelu = nn.GELU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id,attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        
        # Adding a new hidden layer to the output of BERT model
        linear_output1 = self.linear(dropout_output)
        linear_output1 = self.gelu(linear_output1)
        linear_output1 = self.dropout(linear_output1)
        
        # Adding a new hidden layer to the output of BERT model
        linear_output2 = self.linear1(linear_output1)
        linear_output2 = self.gelu(linear_output2)
        linear_output2 = self.dropout(linear_output2)
        
        # Adding the sigmoid layer for the model output
        linear_output3 = self.linear2(linear_output2)
        final_layer = self.sigmoid(linear_output3)
        
        return final_layer

In [12]:
from torch.optim import Adam
from tqdm.notebook import tqdm
from alive_progress import alive_bar
import time

In [13]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

1.13.1+cu117
11.7
8500
NVIDIA GeForce RTX 3060 Laptop GPU
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Laptop GPU', major=8, minor=6, total_memory=6143MB, multi_processor_count=30)


In [14]:
torch.cuda.current_device()

0

# Train

In [15]:
%%time
torch.backends.cudnn.benchmark = True

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    
    train_dataloader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle=False) #, num_workers=4)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size = 64) #, num_workers = 4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
            
    # model, optimizer, dataloader = accelerator.prepare(model, optimizer, train_dataloader)

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            count = 0
            print(f'EPOCH NUMBER {epoch_num}')
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad(set_to_none = True)
                batch_loss.backward()
                optimizer.step()              
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in tqdm(val_dataloader):
                    count += 1
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}'
            )
            
    return model
                  
EPOCHS = 5
model = BertClassifier()
LR = 8e-6
              
final_model = train(model, (train_df), (test_df), LR, EPOCHS)

Some weights of the model checkpoint at paulagarciaserrano/roberta-depression-detection were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at paulagarciaserrano/roberta-depression-detection and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

  0%|          | 0/1280000 [00:00<?, ?it/s]

  0%|          | 0/320000 [00:00<?, ?it/s]

EPOCH NUMBER 0


  0%|          | 0/20000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

Epochs: 4 | Train Loss:  0.006                 | Train Accuracy:  0.921                 | Val Loss:  0.007                 | Val Accuracy:  0.874
EPOCH NUMBER 4


  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

Epochs: 5 | Train Loss:  0.006                 | Train Accuracy:  0.925                 | Val Loss:  0.007                 | Val Accuracy:  0.873
CPU times: total: 9h 44min 1s
Wall time: 16h 26min 38s


64 = 38.2s
32 = 42.7s
16 = 58.6s 

# Running the original model on sample of test data

In [15]:
test = pd.read_csv('Dataset/Twitter/test.csv', encoding = "ISO-8859-1", usecols = ["Sentiment","SentimentText"]).rename(columns = {'Sentiment': 'target', 'SentimentText': 'text'})
test['target'] = np.where(test['target']==1, 0, 1)
print(len(test))
test.head()

1578614


,target,text
0,1,is so sad for my APL frie...
1,1,I missed the New Moon trail...
2,0,omg its already 7:30 :O
3,1,.. Omgaga. Im sooo im gunna CRy. I'...
4,1,i think mi bf is cheating on me!!! ...


In [54]:
test1 = test[789000:-789000]

In [55]:
test1['target'].value_counts()

0    381
1    233
Name: target, dtype: int64

In [56]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [57]:
def custom_standardization(input_data):
    import preprocessor as p
    processed_data = p.clean(input_data)
    lowercase_value = processed_data.lower()
    return lowercase_value

In [58]:
test1["text"] = test1["text"].parallel_apply(custom_standardization)

In [63]:
model = BertClassifier()
# model = torch.load('model_BERT_Trained.pkl')

Some weights of the model checkpoint at paulagarciaserrano/roberta-depression-detection were not used when initializing RobertaModel: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at paulagarciaserrano/roberta-depression-detection and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [68]:
def evaluate(model, test_data):
    
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size = 256)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    res = []
    total_acc_test = 0
    with torch.no_grad():
        count = 0
        for test_input, test_label in tqdm(test_dataloader):
            
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            # print(test_data, output)
            res.append(output.argmax(dim=1))

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return res
    
res = evaluate(model, test1)

  0%|          | 0/614 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Test Accuracy:  0.621


# Link to the our trained model:  https://drive.google.com/file/d/1A5OFtNzN8-rq_nk1OpC1ZVy_bE1K24U7/view?usp=share_link